In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,count, isnan,max

spark = SparkSession.builder.appName("Q1").getOrCreate()

df = spark.read.csv("emp_data.csv",header=True,inferSchema=True)

# df.printSchema()

"""
null_check_df = df.select([
    count(when( col(c).isNull() | (col(c)=="") | isnan(c)  , c)).alias(c)
    for c in df.columns
])

null_check_df.show()
"""

df = df.fillna({"LastName":"Unknown"})
df = df.filter(col("EmpID").isNotNull() & col("StartDate").isNotNull() & col("ExitDate").isNotNull())

df = df.withColumn("Current Employee Rating",
when(col("Current Employee Rating")<1,1)
.when(col("Current Employee Rating")>5,5)
.otherwise(col("Current Employee Rating"))
)


""" 
outlier_df = df.select(
    count(when((col("Current Employee Rating")<1) | (col("Current Employee Rating")>5),col("Current Employee Rating")))
)

outlier_df.show()
"""


""" 
duplictes = df.groupBy(df.columns).count().filter("count>1")

duplictes.show()
"""


df.dropDuplicates(["EmpID"])
df.dropDuplicates()


# Q4

deptgroup = df.groupby(["DepartmentType","Title"]).count()
deptgroup.show()


# Q5

highest_rating_dept_df = df.groupBy("DepartmentType").agg(max("Current Employee Rating").alias("Current Employee Rating"))
highest_rating_dept_df.show()


joined_rating_df = df.join(
    highest_rating_dept_df,
    (df["DepartmentType"] == highest_rating_dept_df["DepartmentType"]) & (df["Current Employee Rating"] == highest_rating_dept_df["Current Employee Rating"]),
    "inner"
)

joined_rating_df.show()


df.write.csv("output1",header=True,mode="overwrite")


spark.stop()


24/12/30 07:26:36 WARN Utils: Your hostname, prateek-HP-Pavilion-Laptop-15-eh1xxx resolves to a loopback address: 127.0.1.1; using 10.20.17.8 instead (on interface wlo1)
24/12/30 07:26:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 07:26:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+--------------------+-----+
|      DepartmentType|               Title|count|
+--------------------+--------------------+-----+
|               IT/IS|       Data Analyst |    1|
|               IT/IS|Principal Data Ar...|    3|
|               IT/IS|        Data Analyst|    2|
|   Production       |    Network Engineer|    1|
|               IT/IS|Sr. Network Engineer|   18|
|    Executive Office|    Network Engineer|   15|
|               IT/IS|    Network Engineer|   11|
|               IT/IS|      Data Architect|    2|
|               IT/IS|  Area Sales Manager|    3|
|               Sales|  Area Sales Manager|   62|
|               IT/IS|Enterprise Architect|    4|
|               IT/IS|          IT Support|   23|
|               IT/IS|      Sr. Accountant|    1|
|               IT/IS|             Sr. DBA|    3|
|               IT/IS|Database Administ...|    6|
|               IT/IS|                 CIO|    1|
|   Production       |Production Techni...|   93|


24/12/30 07:26:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+----------+---------+---------+---------+-------------------+-----------------+------------+--------------+------------+-------+--------------------------+---------------+--------------------+--------------------+----------+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+--------------------+-----------------------+
|EmpID| FirstName| LastName|StartDate| ExitDate|              Title|       Supervisor|BusinessUnit|EmployeeStatus|EmployeeType|PayZone|EmployeeClassificationType|TerminationType|      DepartmentType|            Division|       DOB|State|JobFunctionDescription|GenderCode|LocationCode|RaceDesc|MaritalDesc|Performance Score|Current Employee Rating|      DepartmentType|Current Employee Rating|
+-----+----------+---------+---------+---------+-------------------+-----------------+------------+--------------+------------+-------+--------------------------+---------------+--------------------+---------------